![](../images/itam_logo.png)

M. Sc. Liliana Millán Núñez liliana.millan@itam.mx

Noviembre 2020


### K-means

#### Ejemplo

**Segmentación de clientes.** Contamos con un set de datos que contiene las transacciones de una marca inglesa de ropa en línea desde el primero de diciembre del 2010 hasta el 9 de diciembre del 2011. Contamos con el identificador de un producto, la cantidad de productos compradas por transacción, la fecha, y el precio unitario en libras esterlinas. Datos obtenidos de [UCI datasets](http://archive.ics.uci.edu/ml/datasets/Online+Retail).

En `sklearn` ocupamos el modelo `KMeans` que está en el paquete `sklearn.cluster`.

Los hiperparámetros de este algoritmo son:

+ `n_clusters`: Número de centroides
+ `init`: Coordenadas de inicio de los centroides
+ `max_iter`: Número de iteraciones

El método regresa las coordenadas de los centroides obtenidos en el atributo `cluster_centers_`, la etiqueta de a qué grupo pertence cada observación en el atributo `labels_` —permite agregar la etiqueta de grupo a los datos :)—.

$\rightarrow$ como en TODO método que ocupa distancias **DEBEMOS** escalar los datos!!!

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from matplotlib.ticker import FuncFormatter

In [ ]:
def number_formatter(number, pos=None):
    """Convert a number into a human readable format."""
    magnitude = 0
    while abs(number) >= 1000:
        magnitude += 1
        number /= 1000.0
        
    return '%.1f%s' % (number, ['', 'K', 'M', 'B', 'T', 'Q'][magnitude])

In [ ]:
online_retail = pd.read_csv("~/Documents/itam/mineria_datos_licenciatura/data/online_retail.csv")
online_retail.head()

In [ ]:
columns_cleaned = {column: column.lower() for column in online_retail.columns.values}
online_retail.rename(columns=columns_cleaned, inplace=True)

In [ ]:
online_retail.describe()

In [ ]:
a = sns.boxplot(x='quantity', data=online_retail, showmeans=True)
a.set_title("Distribución de Quantity")
a.xaxis.set_major_formatter(FuncFormatter(number_formatter))

In [ ]:
data = online_retail[(online_retail.quantity > 0) & 
                     (online_retail.unitprice > 0)][['quantity', 'unitprice']]

In [ ]:
data.shape

In [ ]:
a = sns.boxplot(x='quantity', data=data, showmeans=True)
a.set_title("Distribución de Quantity")
a.xaxis.set_major_formatter(FuncFormatter(number_formatter))

In [ ]:
a = sns.boxplot(x='unitprice', data=data, showmeans=True)
a.set_title("Distribución de Precio unitario")
a.xaxis.set_major_formatter(FuncFormatter(number_formatter))
a.set_xlim(0, 500)

In [ ]:
data.describe()

In [ ]:
filtered_data = data[(data.unitprice < 501) & (data.quantity < 101)]

In [ ]:
filtered_data.shape

In [ ]:
filtered_data.describe()

In [ ]:
from sklearn.preprocessing import scale

In [ ]:
retail_scaled = scale(filtered_data, with_mean=True, with_std=True)
retail_scaled

¡¡¡Ocupemos K-Means!!!

In [ ]:
from sklearn.cluster import KMeans

np.random.seed(200427)

kmeans = KMeans(n_clusters=4, max_iter=10)
k_means_4 = kmeans.fit(retail_scaled)
k_means_4

¿Cuántas iteraciones ocupamos?

In [ ]:
k_means_4.n_iter_

Obtengamos las coordenadas de cada cluster.

In [ ]:
k_means_4.cluster_centers_

Obtengamos a qué grupo corresponde cada observación

In [ ]:
labels = k_means_4.labels_
labels[:10]

Juntemos las observaciones y los grupos en los que se encuentran


In [ ]:
data = pd.DataFrame({'quantity': retail_scaled[:,0], 
                     'unitprice': retail_scaled[:,1],
                     'group': labels})

data.head()

Veamos de qué tamaño es cada cluster


In [ ]:
data.groupby(['group'])['quantity']\
.count()\
.reset_index()\
.rename(columns={"quantity": "count"})

Visualicemos las observaciones y los centroides


In [ ]:
centroids = k_means_4.cluster_centers_
centroids

In [ ]:
data_w_centroids = pd.DataFrame({'quantity': centroids[:,0],
                                 'unitprice': centroids[:,1],
                                 'group': ['C','C','C','C']})

data_w_centroids

In [ ]:
all_data = data.append(data_w_centroids)

In [ ]:
sns.scatterplot(x='quantity', y='unitprice', hue="group", data=all_data)

Visualización en escala original de cantidad y precio unitario

In [ ]:
data = pd.DataFrame({'quantity': filtered_data.quantity, 
                     'unitprice': filtered_data.unitprice,
                     'group': labels})

data.head()

Tendremos que reescalar las coordenadas de los centroides

In [ ]:
# quantity
mean_quantity = filtered_data.quantity.mean()
std_quantity = filtered_data.quantity.std()
# unitprice
mean_unitprice = filtered_data.unitprice.mean()
std_unitprice = filtered_data.unitprice.std()

data_w_centroids_escaled = pd.DataFrame({'quantity': data_w_centroids.quantity * 
                                         std_quantity + mean_quantity,
                                        'unitprice': data_w_centroids.unitprice * 
                                        std_unitprice + mean_unitprice,
                                        'group': data_w_centroids.group})


In [ ]:
data_w_centroids_escaled

In [ ]:
all_data = data.append(data_w_centroids_escaled)

In [ ]:
sns.scatterplot(x='quantity', y='unitprice', hue="group", data=all_data)

Regresemos a la pregunta: ¿Cómo sabemos cuántos grupos están bien?

+ Dominio del negocio —expertise— 
+ Al ir aumentando $k$ lo que vamos haciendo es disminuir la cantidad de error en el *cluster* —disminuir la varianza—, en el caso extremo cada punto es un *cluster* con 0 error, lo que implica sobreajustar los datos. 

**Método del codo (*Elbow method*)**

En este método se utiliza la varianza explicada con el número de *clusters* generado, si al agregar un nuevo *cluster* no se mejora "mucho" la varianza explicada, entonces no vale la pena agregarlo. Conforme se van agregando *clusters* la explicación de la varianza —ganancia de información— disminuye, cuando esta ganacia es marginal se genera una especie de "codo" en la gráfica (\# clusters - % variaza explicada). **Para obtener el porcentaje de varianza explicada se divide la varianza del *cluster* entre la varianza total de los datos**, o bien teniendo el SSE (sum of squared errors) de cada cluster, mientras menor sea el SSE mayor información.

El SSE en `sklearn` se obtiene del atributo `inertia_` de un objeto `KMeans`, por lo que podemos hacer varios modelos de `Kmeans` con diferentes números de grupos y graficar el SSE para obtener "el codo" y tomar una decisión. 

In [ ]:
k_means_results = []

for k in [3,4,5,6,7,9,10]:
    kmeans = KMeans(n_clusters=k, max_iter=50)
    k_means_results.append(kmeans.fit(retail_scaled))

In [ ]:
sses = pd.DataFrame({'k': [3,4,5,6,7,9,10],
                     'sse': [round(k_means_results[0].inertia_,2),
                             round(k_means_results[1].inertia_,2), 
                             round(k_means_results[2].inertia_,2),
                             round(k_means_results[3].inertia_,2), 
                             round(k_means_results[4].inertia_,2),
                             round(k_means_results[5].inertia_,2),
                             round(k_means_results[6].inertia_,2)]})

sses

**Gráfica de codo** 

In [ ]:
plt.clf()
plt.plot(sses.k, sses.sse)
plt.scatter(sses.k, sses.sse)
plt.xticks([3,4,5,6,7,9,10])
plt.xlabel("k")
plt.ylabel("sse")
plt.title("Elbow graph")
plt.show()

In [ ]:
k_means_results[4].n_iter_

Con 4 clusters se encuentra el *codo* sin embargo se identifica que con 5 o 6 *clusters* todavía hay una disminución en SSE, en este caso podemos corroborar con los expertos de negocio para identificar qué número de *clusters* puede ser más conveniente. 

#### Desventajas

+ Saber el número de grupos *apriori*. A veces es muy costoso correr k-means con diferente número de *clusters*
+ La inicialización aleatoria de los centroides cambia el resultado final y por lo tanto los grupos formados
+ Un dato puede formar parte sólo de un grupo
+ Sólo aplica a datos numéricos
+ No es bueno para trabajar con datos ruidosos o con outliers
+ No es apropiado para conjuntos de datos no lineales debido a que hace *esferas* a partir de los centroides. Los datos mostrados en la siguiente gráfica (tomada de [github dgrtwo](https://github.com/dgrtwo/dgrtwo.github.com/blob/master/_R/2015-01-16-kmeans-free-lunch.Rmd)) no pueden ser separados en 2 *clusters* debido a que una *esfera* esta contenida en la otra. 

![](../images/non_linear_kmeans.png)
<br>
Fuente: [Cross validated](https://stats.stackexchange.com/questions/133656/how-to-understand-the-drawbacks-of-k-means)

K-Means haría algo asi: 

![](../images/non_linear_kmeans_2.png)
<br>
Fuente: [Cross validated](https://stats.stackexchange.com/questions/133656/how-to-understand-the-drawbacks-of-k-means)

Existen algoritmos de agrupación jerárquicos que clasificarían correctamente las observaciones: 

![](../images/non_linear_data_hierarchical_clustering.png)
<br>
Fuente: [Cross validated](https://stats.stackexchange.com/questions/133656/how-to-understand-the-drawbacks-of-k-means)